# Introduction

**Welcome to team JAKK's final notebook!**
In this project, we wanted to find the BEST neighborhood in Pittsburgh. To do that, we used a metric based on one of America's biggest symbols of freedom: the vehicle.

In our project, we looked at 4 different datasets for 4 different submetrics:
* Police Incidents
* Traffic count
* Bridges
* Speed Hump

All of these relate to vehicular transportation in some shape or form, which we will explain later. We thought of this metric because when a neighborhood is easily able to be traveled through, then that means that it is stress free and leisurely. Additionally, there was a lot of data on metrics that related to vehicular transportation. 

Another metric that we considered pursuing would be a neighborhood's orientation towards nature, which includes parks and greenspaces, but the datasets didn't work out as well for that one.

# The Metric

Our Metric: **Feasabiltiy of Vehicles!**
Our Datasets (plus an explaination):
* Police Incidents
    * The higher the amount of police incidents, the harder it might be to travel through an area. Additionally, the more likely that your car will be robbed.
* Traffic count
    * The higher the traffic count, the harder it will be to traverse through a neighborhood. Additionally, more traffic will indicate a noisier, more congested neighborhood.
* Bridges
    * Pittsburgh is known for its Bridges. Bridges are nice looking. Bridges allow vehicles to go over water channels. More bridges are good.
* Speed Hump
    * Speed humps could be an indicator of a residential area. More speed humps impede the traffic flow. We want smooth sailling, so less speed humps is better!

Overall, this metric uses the lens of the vehicle to see Pittsburgh through the eyes of its transportation. By doing this, it includes scenery through bridges, congestion through traffic, safety through police incidents, and the ability to go fast through speed humps. 

# The Best Neighborhood

INSERT CODE, ANALYSIS...

# Conclusion
